In [11]:
# Data visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import glob
# Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [34]:
files = glob.glob('data/new_sensors/*.txt')
feature_files = files[:-1]
data = []

for i, f in enumerate(feature_files):
    label = f.split(',')[6]
    csv = pd.read_csv(f, header=None)
    for item in csv.values[50:-50]:
        item = np.append(item, label)
        data.append(item)
        


In [35]:
data

[array(['0.13714981', '-0.7908573', '9.961029', 'Resting'], dtype='<U32'),
 array(['0.12517166', '-0.7884617', '9.967018', 'Resting'], dtype='<U32'),
 array(['0.13415527', '-0.7941513', '9.970013', 'Resting'], dtype='<U32'),
 array(['0.13535309', '-0.79265404', '9.9646225', 'Resting'], dtype='<U32'),
 array(['0.12097931', '-0.7875633', '9.970612', 'Resting'], dtype='<U32'),
 array(['0.13535309', '-0.7878628', '9.967018', 'Resting'], dtype='<U32'),
 array(['0.13535309', '-0.7998409', '9.964922', 'Resting'], dtype='<U32'),
 array(['0.12157822', '-0.80043983', '9.97151', 'Resting'], dtype='<U32'),
 array(['0.123075485', '-0.79774475', '9.961927', 'Resting'], dtype='<U32'),
 array(['0.13295746', '-0.79325294', '9.968216', 'Resting'], dtype='<U32'),
 array(['0.12097931', '-0.79445076', '9.964024', 'Resting'], dtype='<U32'),
 array(['0.11918259', '-0.7863655', '9.965521', 'Resting'], dtype='<U32'),
 array(['0.13355637', '-0.79205513', '9.969114', 'Resting'], dtype='<U32'),
 array(['0.1248722

In [37]:
columns = ['x', 'y', 'z', 'activity']
data = pd.DataFrame(data = data, columns=columns)
data.head()

,x,y,z,activity
0,0.13714981,-0.7908573,9.961029,Resting
1,0.12517166,-0.7884617,9.967018,Resting
2,0.13415527,-0.7941513,9.970013,Resting
3,0.13535309,-0.79265404,9.9646225,Resting
4,0.12097931,-0.7875633,9.970612,Resting


In [38]:
data['activity'].value_counts()

Stair climber    134022
Jogging           12526
Fast Walking      10055
Resting            8825
Name: activity, dtype: int64

In [20]:
data = np.array(data, dtype=object)

In [5]:
len(data)

165428

In [6]:
data[165427]

array(['0.36521912', '-0.42425537', '0.02319336', 'Stair climber'],
      dtype=object)

In [7]:
X = data[:,0:3].astype(float)
y = data[:,3]

In [17]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

y_cat = to_categorical(encoded_y)
y_cat[0]


array([0., 0., 0., 1.], dtype=float32)

In [8]:
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.reshape(-1, 1))


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y_ohe, test_size=0.3)

In [85]:
model = Sequential()
model.add(Dense(8, input_dim=3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(4, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
model.fit(x_train, y_train, epochs = 50, shuffle = True, batch_size=10, verbose=2)

Epoch 1/50
11580/11580 - 10s - loss: 0.4949 - accuracy: 0.8307
Epoch 2/50
11580/11580 - 9s - loss: 0.3991 - accuracy: 0.8378
Epoch 3/50
11580/11580 - 9s - loss: 0.3783 - accuracy: 0.8393
Epoch 4/50
11580/11580 - 9s - loss: 0.3623 - accuracy: 0.8444
Epoch 5/50
11580/11580 - 9s - loss: 0.3593 - accuracy: 0.8466
Epoch 6/50
11580/11580 - 9s - loss: 0.3584 - accuracy: 0.8475
Epoch 7/50
11580/11580 - 9s - loss: 0.3572 - accuracy: 0.8486
Epoch 8/50
11580/11580 - 9s - loss: 0.3512 - accuracy: 0.8501
Epoch 9/50
11580/11580 - 9s - loss: 0.3495 - accuracy: 0.8507
Epoch 10/50
11580/11580 - 9s - loss: 0.3483 - accuracy: 0.8501
Epoch 11/50
11580/11580 - 9s - loss: 0.3473 - accuracy: 0.8500
Epoch 12/50
11580/11580 - 9s - loss: 0.3531 - accuracy: 0.8514
Epoch 13/50
11580/11580 - 10s - loss: 0.3503 - accuracy: 0.8508
Epoch 14/50
11580/11580 - 9s - loss: 0.3490 - accuracy: 0.8503
Epoch 15/50
11580/11580 - 9s - loss: 0.3458 - accuracy: 0.8501
Epoch 16/50
11580/11580 - 9s - loss: 0.3451 - accuracy: 0.8511

In [87]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc * 100)

1551/1551 - 1s - loss: 0.3389 - accuracy: 0.8513

Test accuracy: 85.12764573097229


In [88]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [89]:
predictions = probability_model.predict(x_test)

In [96]:
y_test[0]

array([0., 0., 0., 1.])

In [108]:
predictions[0]

array([0.15248616, 0.15694872, 0.15295918, 0.1510499 , 0.3865561 ],
      dtype=float32)

In [91]:
prediction = np.argmax(predictions[0])
prediction

3

In [84]:
data_list = pd.DataFrame(data)
data_list.to_csv('test.csv', index=None, header=None)

In [93]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

inv = ohe.inverse_transform(y_ohe).ravel()
labels = np.unique(inv)
print(labels[3])

Stair climber


In [97]:
model.save('ml_model')

INFO:tensorflow:Assets written to: ml_model\assets
